In [1]:
from datasets import load_dataset
import requests
import pandas as pd
from tqdm import tqdm
import os
import json
import re
from urllib.parse import urlparse
import glob
import time
import numpy as np

In [2]:
token = "INSERT_TOKEN"
headers = {"Authorization": f"token {token}"}

In [3]:
OUTPUT_CSV = r'output_files/fix_prs.csv'

os.makedirs(os.path.dirname(OUTPUT_CSV), exist_ok=True)

if os.path.exists(OUTPUT_CSV):
    # --- CAMINHO 1: O ARQUIVO JÁ EXISTE ---
    print(f"Arquivo {OUTPUT_CSV} encontrado. Lendo dados do disco...")
    all_fix_prs = pd.read_csv(OUTPUT_CSV)

else:
    # --- CAMINHO 2: O ARQUIVO NÃO EXISTE ---
    print(f"Arquivo {OUTPUT_CSV} não encontrado.")
    # --- Seu código original para carregar e processar ---
    aidev_pop = load_dataset("hao-li/AIDev", "pull_request")
    pandas_aidev_pop = aidev_pop['train'].to_pandas()
    
    task_types = load_dataset("hao-li/AIDev", "pr_task_type")
    pandas_task_types = task_types['train'].to_pandas()
    pandas_task_types.rename(columns={'confidence': 'type_confidence'}, inplace=True)
    aidev_pop_with_types = pd.merge(pandas_aidev_pop, pandas_task_types[['id','type','type_confidence']], on='id')

    human_prs = load_dataset("hao-li/AIDev", "human_pull_request")
    pandas_human_prs = human_prs['train'].to_pandas()

    human_task_types = load_dataset("hao-li/AIDev", "human_pr_task_type")
    pandas_human_task_types = human_task_types['train'].to_pandas()
    pandas_human_task_types.rename(columns={'confidence': 'type_confidence'}, inplace=True)
    human_prs_with_types = pd.merge(pandas_human_prs, pandas_human_task_types[['id','type']], on='id')
    fix_human_prs = human_prs_with_types[human_prs_with_types['type'] == 'fix']
    fix_agent_prs = aidev_pop_with_types[aidev_pop_with_types['type'] == 'fix']
    
    cols_to_keep = ['id','number','user','user_id','agent','title','body','state',
                    'created_at','closed_at','merged_at','repo_url','html_url']
    
    all_fix_prs = pd.concat([
        fix_human_prs[cols_to_keep],
        fix_agent_prs[cols_to_keep]
    ])
    
    try:
        all_fix_prs.to_csv(OUTPUT_CSV, index=False)
    except Exception as e:
        print(f"ERRO: Falha ao salvar o arquivo CSV. Erro: {e}")
all_fix_prs

Arquivo output_files/fix_prs.csv encontrado. Lendo dados do disco...


,id,number,user,user_id,agent,title,body,state,created_at,closed_at,merged_at,repo_url,html_url
0,2438086945,88748,iamrajjoshi,33237075,Human,:bug: fix: update how we fetch workflow_id and...,i realized i made a mistake for how i fetch th...,closed,2025-04-03T21:36:59Z,2025-04-04T15:10:57Z,2025-04-04T15:10:57Z,https://api.github.com/repos/getsentry/sentry,https://github.com/getsentry/sentry/pull/88748
1,2265431531,83085,ArthurKnaus,7033940,Human,fix(org-stats): Require project membership,### Problem\r\n\r\nIf the user is not member o...,closed,2025-01-08T07:47:13Z,2025-01-08T08:49:40Z,2025-01-08T08:49:40Z,https://api.github.com/repos/getsentry/sentry,https://github.com/getsentry/sentry/pull/83085
2,2622011651,94465,bukzor,640328,Human,fix(dev): mktemp: too few X's in template,"For maximum compatibility, busybox mktemp requ...",closed,2025-06-26T18:54:10Z,2025-06-26T19:57:23Z,2025-06-26T19:57:23Z,https://api.github.com/repos/getsentry/sentry,https://github.com/getsentry/sentry/pull/94465
3,2565399631,92785,dashed,139499,Human,fix(billing): Update calculateCategoryPrepaidU...,Closes https://linear.app/getsentry/issue/BIL-...,closed,2025-06-03T22:22:51Z,2025-06-05T18:13:54Z,2025-06-05T18:13:54Z,https://api.github.com/repos/getsentry/sentry,https://github.com/getsentry/sentry/pull/92785
4,2374801945,86438,brendanhsentry,171613822,Human,fix: copy updates to checkout page,closes https://github.com/getsentry/getsentry/...,closed,2025-03-05T22:39:12Z,2025-03-06T16:57:20Z,2025-03-06T16:57:20Z,https://api.github.com/repos/getsentry/sentry,https://github.com/getsentry/sentry/pull/86438
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9879,3260452571,1542,devin-ai-integration[bot],158243242,Devin,Fix CSS color assertions in test_lambdas.py,# Fix CSS color assertions in test_lambdas.py\...,closed,2025-07-24T16:14:38Z,2025-07-25T11:29:31Z,2025-07-25T11:29:31Z,https://api.github.com/repos/reflex-dev/reflex...,https://github.com/reflex-dev/reflex-web/pull/...
9880,2857103111,2151,devin-ai-integration[bot],158243242,Devin,fix: Initialize storage in StringKnowledgeSource,Fixes #2150\n\n## Issue\nStringKnowledgeSource...,closed,2025-02-17T08:16:22Z,2025-02-25T16:39:20Z,NaN,https://api.github.com/repos/crewAIInc/crewAI,https://github.com/crewAIInc/crewAI/pull/2151
9881,2857279950,8459,devin-ai-integration[bot],158243242,Devin,Add missing OpenSSL TLSEXT status response codes,Fixes the build failure in the OpenSSL coexist...,closed,2025-02-17T09:32:13Z,2025-02-17T09:34:11Z,NaN,https://api.github.com/repos/wolfSSL/wolfssl,https://github.com/wolfSSL/wolfssl/pull/8459
9882,2857942945,2,devin-ai-integration[bot],158243242,Devin,fix: improve dark mode input focus and toggle ...,# UI Improvements: Dark Mode Input Focus and T...,closed,2025-02-17T13:57:22Z,2025-02-17T14:40:12Z,2025-02-17T14:40:12Z,https://api.github.com/repos/jina-ai/deepsearc...,https://github.com/jina-ai/deepsearch-ui/pull/2


In [4]:
# Função para extrair possíveis issues do texto
def extract_issue_numbers(text):
    if not text:
        return []
    # Procura padrões como #123 ou owner/repo#123
    pattern = r'(?:\b[\w-]+/[\w-]+)?#(\d+)\b'
    return [int(m) for m in re.findall(pattern, text)]

# Aplicando no dataframe
all_fix_prs['possible_issues'] = all_fix_prs.apply(
    lambda row: extract_issue_numbers(str(row['title']) + ' ' + str(row['body'])),
    axis=1
)
all_fix_prs['possible_issues'] = all_fix_prs['possible_issues'].apply(lambda x: list(set(x)) if isinstance(x, list) else x)

In [5]:
# ----- FUNÇÃO HELPER MODIFICADA -----
def _get_issue_type(url: str, session: requests.Session) -> str:
    """
    Faz uma chamada GET para a API de /issues e determina se 
    é uma Issue pura, um Pull Request, ou não existe.
    Retorna: 'issue', 'pull_request', 'not_found'
    """
    try:
        # MUDANÇA CRUCIAL: session.get() em vez de session.head()
        # Precisamos do corpo JSON para verificar o campo 'pull_request'
        response = session.get(url, timeout=10) 
        
        # 404 Not Found ou 410 Gone = Não existe
        if response.status_code in (404, 410):
            return 'not_found'
        
        # Lidar com Rate Limit (igual ao original)
        if response.status_code == 403 and 'X-RateLimit-Remaining' in response.headers:
            if int(response.headers['X-RateLimit-Remaining']) == 0:
                print("Rate limit atingido. Aguardando 15 minutos...")
                time.sleep(15 * 60)
                return _get_issue_type(url, session) # Tenta de novo

        # Outros erros (500, 403-Forbidden, etc.)
        if response.status_code != 200:
            return 'not_found' # Trata outros erros como 'not_found'

        # --- SUCESSO (200 OK) ---
        data = response.json()
        
        # A VERIFICAÇÃO QUE VOCÊ PEDIU:
        if 'pull_request' in data:
            return 'pull_request' # É um PR
        else:
            return 'issue' # É uma Issue pura

    except requests.exceptions.RequestException:
        return 'not_found' # Erro de rede
        

# ----- FUNÇÃO PRINCIPAL MODIFICADA -----

def validar_issues_do_repo(df: pd.DataFrame, github_token: str) -> pd.DataFrame:
    """
    Recebe um DataFrame com 'repo_url' e 'possible_issues' (lista) 
    e retorna o DataFrame com uma nova coluna 'matched_issues' 
    contendo apenas os números de Issues PURAS (não PRs).
    """
    
    print("Iniciando refatoração: Otimizando chamadas de API...")
    
    # --- 1. SETUP DA SESSÃO E CACHE ---
    
    session = requests.Session()
    session.headers.update({
        'Authorization': f'token {github_token}',
        'Accept': 'application/vnd.github.v3+json',
        'X-GitHub-Api-Version': '2022-11-28'
    })
    
    # Cache armazena o TIPO da issue
    # Formato: {(repo_url, issue_num): str} (ex: 'issue', 'pull_request')
    validation_cache = {}

    # --- 2. ENCONTRAR VERIFICAÇÕES ÚNICAS (Sem mudanças) ---
    
    df_clean = df.dropna(subset=['repo_url', 'possible_issues'])
    df_exploded = df_clean.explode('possible_issues')
    df_exploded = df_exploded.dropna(subset=['possible_issues'])
    df_exploded['possible_issues'] = df_exploded['possible_issues'].astype(int)
    checks_needed = df_exploded[['repo_url', 'possible_issues']].drop_duplicates()
    
    print(f"Otimização: {len(df)} linhas reduzidas a {len(checks_needed)} chamadas de API únicas.")

    # --- 3. EXECUTAR VERIFICAÇÕES DE API (MODIFICADO) ---
    
    for _, row in tqdm(checks_needed.iterrows(), total=len(checks_needed), desc="Validando issues na API"):
        repo_url = row['repo_url']
        issue_num = row['possible_issues']
        cache_key = (repo_url, issue_num)
        
        if issue_num == 0:
            validation_cache[cache_key] = 'not_found' # MUDADO
            continue
            
        api_url = f"{repo_url}/issues/{issue_num}"
        
        # Chama nosso NOVO helper e salva o TIPO no cache
        validation_cache[cache_key] = _get_issue_type(api_url, session) # MUDADO

    # --- 4. MAPEAR RESULTADOS DE VOLTA (MODIFICADO) ---
    
    print("Mapeando resultados de volta para o DataFrame...")
    
    def find_matches(row):
        repo = row['repo_url']
        issues = row['possible_issues']
        
        if not isinstance(issues, list) or pd.isna(repo):
            return [] 
            
        matched_list = []
        for num in issues:
            # Pega o tipo ('issue', 'pull_request', 'not_found') do cache
            cached_result = validation_cache.get((repo, num), 'not_found')
            
            # ADICIONA À LISTA APENAS SE FOR UMA ISSUE PURA
            if cached_result == 'issue': # MUDADO
                matched_list.append(num)
        
        return matched_list

    # Aplica a função de mapeamento (rápida, pois só usa o cache)
    df['matched_issues'] = df.apply(find_matches, axis=1)
    
    print("Refatoração concluída.")
    return df


In [6]:
if False:
    def _check_api_existence(url: str, session: requests.Session) -> bool:
        """
        Função helper que faz uma chamada HEAD (leve) para verificar 
        a existência de um recurso e lida com o rate limit.
        """
        try:
            # Usamos HEAD em vez de GET - não precisamos do corpo, 
            # só do status code. É muito mais rápido.
            response = session.head(url, timeout=10)
            
            # 200 OK = Existe
            if response.status_code == 200:
                return True
            
            # 404 Not Found ou 410 Gone = Não existe
            if response.status_code in (404, 410):
                return False
                
            # Lidar com Rate Limit
            if response.status_code == 403 and 'X-RateLimit-Remaining' in response.headers:
                if int(response.headers['X-RateLimit-Remaining']) == 0:
                    print("Rate limit atingido. Aguardando 15 minutos...")
                    time.sleep(15 * 60) # Espera 15 minutos
                    return _check_api_existence(url, session) # Tenta de novo
            
            # Outros erros (500, etc.)
            return False
            
        except requests.exceptions.RequestException:
            return False
    
    # ----- ESTA É A FUNÇÃO QUE VOCÊ PEDIU -----
    
    def validar_issues_do_repo(df: pd.DataFrame, github_token: str) -> pd.DataFrame:
        """
        Recebe um DataFrame com 'repo_url' e 'possible_issues' (lista) 
        e retorna o DataFrame com uma nova coluna 'matched_issues' 
        contendo apenas os números de issues que realmente existem.
        """
        
        print("Iniciando refatoração: Otimizando chamadas de API...")
        
        # --- 1. SETUP DA SESSÃO E CACHE ---
        
        # Usar uma Session permite reuso de conexão (connection pooling)
        session = requests.Session()
        session.headers.update({
            'Authorization': f'token {github_token}',
            'Accept': 'application/vnd.github.v3+json',
            'X-GitHub-Api-Version': '2022-11-28'
        })
        
        # Cache para armazenar resultados (evita checar a mesma issue 500x)
        # Formato: {(repo_url, issue_num): bool}
        validation_cache = {}
    
        # --- 2. ENCONTRAR VERIFICAÇÕES ÚNICAS (O "X" da otimização) ---
        
        # Limpa linhas onde não temos os dados necessários
        df_clean = df.dropna(subset=['repo_url', 'possible_issues'])
        
        # 'Explode' transforma a lista [1, 2] em duas linhas: (repo, 1) e (repo, 2)
        df_exploded = df_clean.explode('possible_issues')
        
        # Remove casos onde 'possible_issues' era uma lista vazia []
        df_exploded = df_exploded.dropna(subset=['possible_issues'])
        
        # Garante que os números são inteiros
        df_exploded['possible_issues'] = df_exploded['possible_issues'].astype(int)
        
        # Pega apenas as combinações únicas de (repo, issue)
        checks_needed = df_exploded[['repo_url', 'possible_issues']].drop_duplicates()
        
        print(f"Otimização: {len(df)} linhas reduzidas a {len(checks_needed)} chamadas de API únicas.")
    
        # --- 3. EXECUTAR VERIFICAÇÕES DE API EM LOTE ---
        
        for _, row in tqdm(checks_needed.iterrows(), total=len(checks_needed), desc="Validando issues na API"):
            repo_url = row['repo_url']
            issue_num = row['possible_issues']
            cache_key = (repo_url, issue_num)
            
            # Issue 0 não existe no GitHub, podemos pular a API
            if issue_num == 0:
                validation_cache[cache_key] = False
                continue
                
            # Monta a URL da API da issue
            api_url = f"{repo_url}/issues/{issue_num}"
            
            # Chama nosso helper robusto e salva no cache
            validation_cache[cache_key] = _check_api_existence(api_url, session)
    
        # --- 4. MAPEAR RESULTADOS DE VOLTA (Rápido, sem API) ---
        
        print("Mapeando resultados de volta para o DataFrame...")
        
        def find_matches(row):
            repo = row['repo_url']
            issues = row['possible_issues']
            
            # Lida com linhas que tinham dados ruins (NaN)
            if not isinstance(issues, list) or pd.isna(repo):
                return [] 
                
            matched_list = []
            for num in issues:
                # Pega o resultado do cache. 
                # .get(..., False) retorna False se a issue nunca foi checada
                if validation_cache.get((repo, num), False):
                    matched_list.append(num)
            
            return matched_list
    
        # Aplica a função de mapeamento (rápida, pois só usa o cache)
        df['matched_issues'] = df.apply(find_matches, axis=1)
        
        print("Refatoração concluída.")
        return df

In [7]:
OUTPUT_PARQUET = r'output_files\possibles_issues.parquet'
if os.path.exists(OUTPUT_PARQUET):
    # --- CAMINHO 1: O ARQUIVO JÁ EXISTE ---
    print(f"Arquivo {OUTPUT_PARQUET} encontrado. Lendo dados do disco...")
    
    df_com_matches = pd.read_parquet(OUTPUT_PARQUET)
    
    # Adiciona a limpeza de colunas de lista (o problema do numpy.ndarray)
    def clean_parquet_list(item):
        if isinstance(item, list):
            return item
        if isinstance(item, np.ndarray):
            return item.tolist() # Converte array NumPy em lista
        return [] # Converte None, NaN, etc., em lista vazia
    # Limpa as colunas que sabemos que são listas
    list_cols = ['possible_issues', 'matched_issues']
    for col in list_cols:
        if col in df_com_matches.columns:
            df_com_matches[col] = df_com_matches[col].apply(clean_parquet_list)

else:
    # --- CAMINHO 2: O ARQUIVO NÃO EXISTE ---
    print(f"Arquivo {OUTPUT_PARQUET} não encontrado.")
    print("Iniciando a validação de issues pela API (pode demorar)...")
    df_com_matches = validar_issues_do_repo(all_fix_prs, token)
    print(f"Processamento da API concluído. Salvando resultados em {OUTPUT_PARQUET}...")
    try:
        # Garante que o diretório exista
        os.makedirs(os.path.dirname(OUTPUT_PARQUET), exist_ok=True) 
        df_com_matches.to_parquet(OUTPUT_PARQUET, index=False)
        print("Arquivo salvo com sucesso.")
    except Exception as e:
        print(f"ERRO: Falha ao salvar o arquivo Parquet. Erro: {e}")
df_com_matches[['repo_url', 'possible_issues', 'matched_issues']].head()

Arquivo output_files\possibles_issues.parquet encontrado. Lendo dados do disco...


,repo_url,possible_issues,matched_issues
0,https://api.github.com/repos/getsentry/sentry,[],[]
1,https://api.github.com/repos/getsentry/sentry,[],[]
2,https://api.github.com/repos/getsentry/sentry,[],[]
3,https://api.github.com/repos/getsentry/sentry,[],[]
4,https://api.github.com/repos/getsentry/sentry,[],[]


In [8]:
PR_BATCH_SIZE = 50 
PARQUET_FILE = r'output_files\linked_issues.parquet' 
CHECKPOINT_FILE = r'output_files\linked_issues_checkpoint.json'
# --- FUNÇÃO HELPER 1: Gerador de Sub-Query (Sem mudanças) ---
def generate_pr_sub_query(pr_number: int) -> str:
    """
    Cria a parte da query GraphQL para um único PR usando um alias.
    Ex: pr123: pullRequest(number: 123) { ... }
    """
    return f"""
    pr{pr_number}: pullRequest(number: {pr_number}) {{
      closingIssuesReferences(first: 10) {{
        nodes {{
          number
        }}
      }}
    }}
    """

# --- FUNÇÃO HELPER 2: A Função de API Otimizada (Sem mudanças) ---
def get_linked_issues_batched(owner: str, repo: str, pr_numbers: list, headers: dict) -> dict:
    """
    Busca issues linkadas para um LOTE de números de PRs 
    de um ÚNICO repositório em UMA chamada GraphQL.
    """
    graphql_url = "https://api.github.com/graphql"
    
    # 1. Construir a query dinâmica
    sub_queries = "\n".join([generate_pr_sub_query(n) for n in pr_numbers])
    query = f"""
    query ($owner: String!, $repo: String!) {{
      repository(owner: $owner, name: $repo) {{
        {sub_queries}
      }}
    }}
    """
    
    variables = {"owner": owner, "repo": repo}
    
    # Dicionário para armazenar {pr_number: [issue_list]}
    results = {}
    
    try:
        r = requests.post(graphql_url, json={"query": query, "variables": variables}, headers=headers, timeout=30)
        r.raise_for_status()
        data = r.json()
        
        # 2. Lidar com erros parciais do GraphQL
        if 'errors' in data:
            print(f"   Erro de GraphQL em {owner}/{repo}: {data['errors'][0]['message'][:100]}...")
            for num in pr_numbers:
                results[num] = [] 
            return results
            
        repo_data = data.get('data', {}).get('repository')
        if not repo_data:
            raise Exception("'repository' não encontrado na resposta.")

        # 3. Parsear a resposta
        for pr_alias, pr_data in repo_data.items():
            pr_num = int(pr_alias[2:]) 
            
            if pr_data is None:
                results[pr_num] = []
                continue
                
            nodes = pr_data.get('closingIssuesReferences', {}).get('nodes', [])
            results[pr_num] = [n['number'] for n in nodes]
            
        return results

    except Exception as e:
        print(f"   Exceção de Rede/Timeout em {owner}/{repo} (Lote começando com PR {pr_numbers[0]}): {e}")
        for num in pr_numbers:
            results[num] = []
        return results

# --- LÓGICA PRINCIPAL OTIMIZADA (MODIFICADA) ---

# 1. Checa se o ARQUIVO PARQUET existe
if not os.path.exists(PARQUET_FILE):
    print("Iniciando busca otimizada de issues linkadas...")
    
    if os.path.exists(CHECKPOINT_FILE):
        print(f"Carregando progresso do checkpoint: {CHECKPOINT_FILE}")
        with open(CHECKPOINT_FILE, 'r', encoding='utf-8') as f:
            repo_results_map = json.load(f)
    else:
        repo_results_map = {}

    repos_para_processar = df_com_matches[
        ~df_com_matches['repo_url'].isin(repo_results_map.keys())
    ]
    grouped = repos_para_processar.groupby('repo_url')
    
    print(f"Total de repositórios a processar: {len(grouped)}")
    pbar = tqdm(grouped, desc="Processando Repositórios")

    for repo_url, group in pbar:
        try:
            owner_repo = "/".join(repo_url.rstrip("/").split("/")[-2:])
            owner, repo = owner_repo.split("/")
            pbar.set_postfix_str(f"{owner}/{repo}")
        except Exception as e:
            print(f"Erro ao parsear URL do repo {repo_url}: {e}. Pulando...")
            repo_results_map[repo_url] = {} 
            continue
        
        pr_numbers = group['number'].unique().tolist()
        repo_batch_results = {} 

        for i in range(0, len(pr_numbers), PR_BATCH_SIZE):
            batch_pr_numbers = pr_numbers[i:i+PR_BATCH_SIZE]
            batch_results = get_linked_issues_batched(owner, repo, batch_pr_numbers, headers)
            repo_batch_results.update(batch_results)

        repo_results_map[repo_url] = repo_batch_results
        try:
            with open(CHECKPOINT_FILE, 'w', encoding='utf-8') as f:
                json.dump(repo_results_map, f, indent=2)
        except Exception as e:
            print(f"ALERTA: Falha ao salvar checkpoint: {e}")

    # --- FIM DO LOOP ---
    
    print("Processamento de API concluído. Mapeando resultados de volta ao DataFrame...")

    def map_results(row):
        return repo_results_map.get(row['repo_url'], {}).get(row['number'], [])

    df_com_matches['linked_issues'] = df_com_matches.apply(map_results, axis=1)
    
    # 7. Salvar o PARQUET final (MODIFICADO)
    print(f"Salvando resultados em {PARQUET_FILE}...")
    # Parquet salva a lista [1, 2] como uma lista, não como a string "[1, 2]"
    df_com_matches.to_parquet(PARQUET_FILE, index=False)
    
    # Limpar o checkpoint
    if os.path.exists(CHECKPOINT_FILE):
        os.remove(CHECKPOINT_FILE)

    df_com_issues_linkadas = df_com_matches.copy()
else:
    # --- LEITURA DO PARQUET (MUITO MAIS SIMPLES) ---
    print(f"Arquivo {PARQUET_FILE} já existe, lendo do disco.")
    df_com_issues_linkadas = pd.read_parquet(PARQUET_FILE)
    
    def clean_parquet_list(item):
        # Caso 1: É uma lista Python (raro, mas pode acontecer)
        if isinstance(item, list):
            return item
            
        # Caso 2: É um array NumPy (o mais provável)
        if isinstance(item, np.ndarray):
            return item.tolist() # <-- Converte o array em uma lista Python
            
        # Caso 3: É None, NaN, ou qualquer outra coisa
        return [] # Retorna lista vazia

    df_com_issues_linkadas['linked_issues'] = df_com_issues_linkadas['linked_issues'].apply(clean_parquet_list)

df_com_issues_linkadas

Arquivo output_files\linked_issues.parquet já existe, lendo do disco.


,id,number,user,user_id,agent,title,body,state,created_at,closed_at,merged_at,repo_url,html_url,possible_issues,matched_issues,linked_issues
0,2438086945,88748,iamrajjoshi,33237075,Human,:bug: fix: update how we fetch workflow_id and...,i realized i made a mistake for how i fetch th...,closed,2025-04-03T21:36:59Z,2025-04-04T15:10:57Z,2025-04-04T15:10:57Z,https://api.github.com/repos/getsentry/sentry,https://github.com/getsentry/sentry/pull/88748,[],[],[]
1,2265431531,83085,ArthurKnaus,7033940,Human,fix(org-stats): Require project membership,### Problem\r\n\r\nIf the user is not member o...,closed,2025-01-08T07:47:13Z,2025-01-08T08:49:40Z,2025-01-08T08:49:40Z,https://api.github.com/repos/getsentry/sentry,https://github.com/getsentry/sentry/pull/83085,[],[],[82998]
2,2622011651,94465,bukzor,640328,Human,fix(dev): mktemp: too few X's in template,"For maximum compatibility, busybox mktemp requ...",closed,2025-06-26T18:54:10Z,2025-06-26T19:57:23Z,2025-06-26T19:57:23Z,https://api.github.com/repos/getsentry/sentry,https://github.com/getsentry/sentry/pull/94465,[],[],[]
3,2565399631,92785,dashed,139499,Human,fix(billing): Update calculateCategoryPrepaidU...,Closes https://linear.app/getsentry/issue/BIL-...,closed,2025-06-03T22:22:51Z,2025-06-05T18:13:54Z,2025-06-05T18:13:54Z,https://api.github.com/repos/getsentry/sentry,https://github.com/getsentry/sentry/pull/92785,[],[],[]
4,2374801945,86438,brendanhsentry,171613822,Human,fix: copy updates to checkout page,closes https://github.com/getsentry/getsentry/...,closed,2025-03-05T22:39:12Z,2025-03-06T16:57:20Z,2025-03-06T16:57:20Z,https://api.github.com/repos/getsentry/sentry,https://github.com/getsentry/sentry/pull/86438,[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9879,3260452571,1542,devin-ai-integration[bot],158243242,Devin,Fix CSS color assertions in test_lambdas.py,# Fix CSS color assertions in test_lambdas.py\...,closed,2025-07-24T16:14:38Z,2025-07-25T11:29:31Z,2025-07-25T11:29:31Z,https://api.github.com/repos/reflex-dev/reflex...,https://github.com/reflex-dev/reflex-web/pull/...,[],[],[]
9880,2857103111,2151,devin-ai-integration[bot],158243242,Devin,fix: Initialize storage in StringKnowledgeSource,Fixes #2150\n\n## Issue\nStringKnowledgeSource...,closed,2025-02-17T08:16:22Z,2025-02-25T16:39:20Z,None,https://api.github.com/repos/crewAIInc/crewAI,https://github.com/crewAIInc/crewAI/pull/2151,[2150],[2150],[2150]
9881,2857279950,8459,devin-ai-integration[bot],158243242,Devin,Add missing OpenSSL TLSEXT status response codes,Fixes the build failure in the OpenSSL coexist...,closed,2025-02-17T09:32:13Z,2025-02-17T09:34:11Z,None,https://api.github.com/repos/wolfSSL/wolfssl,https://github.com/wolfSSL/wolfssl/pull/8459,[],[],[]
9882,2857942945,2,devin-ai-integration[bot],158243242,Devin,fix: improve dark mode input focus and toggle ...,# UI Improvements: Dark Mode Input Focus and T...,closed,2025-02-17T13:57:22Z,2025-02-17T14:40:12Z,2025-02-17T14:40:12Z,https://api.github.com/repos/jina-ai/deepsearc...,https://github.com/jina-ai/deepsearch-ui/pull/2,[],[],[]


In [9]:
key_cols = ['id', 'number', 'repo_url']
df_final = df_com_issues_linkadas.copy()
df_final = df_final.set_index(key_cols)
df_matched_indexed = df_com_matches.set_index(key_cols)

df_final.update(df_matched_indexed[['possible_issues','matched_issues']])

df_final = df_final.reset_index()

df_final['issues'] = df_final.apply(
    lambda row: list(set(row['matched_issues']) | set(row['linked_issues'])),
    axis=1
)
prs_com_issue = df_final[df_final['issues'].str.len() > 0]
prs_com_issue

C:\Users\USER\AppData\Local\Temp\ipykernel_25040\3270513541.py:6: PerformanceWarning: indexing past lexsort depth may impact performance.
  df_final.update(df_matched_indexed[['possible_issues','matched_issues']])
C:\Users\USER\AppData\Local\Temp\ipykernel_25040\3270513541.py:6: PerformanceWarning: indexing past lexsort depth may impact performance.
  df_final.update(df_matched_indexed[['possible_issues','matched_issues']])


,id,number,repo_url,user,user_id,agent,title,body,state,created_at,closed_at,merged_at,html_url,possible_issues,matched_issues,linked_issues,issues
1,2265431531,83085,https://api.github.com/repos/getsentry/sentry,ArthurKnaus,7033940,Human,fix(org-stats): Require project membership,### Problem\r\n\r\nIf the user is not member o...,closed,2025-01-08T07:47:13Z,2025-01-08T08:49:40Z,2025-01-08T08:49:40Z,https://github.com/getsentry/sentry/pull/83085,[],[],[82998],[82998]
7,2563038011,76,https://api.github.com/repos/mediar-ai/terminator,soheil-01,161890530,Human,fix: reorder selector match patterns to priori...,#70,closed,2025-06-03T08:07:58Z,2025-06-03T15:07:07Z,2025-06-03T15:07:07Z,https://github.com/mediar-ai/terminator/pull/76,[70],[70],[],[70]
17,2257483414,19989,https://api.github.com/repos/risingwavelabs/ri...,zwang28,70626450,Human,fix(meta): resolve deadlock caused by Hummock ...,I hereby agree to the terms of the [RisingWave...,closed,2025-01-02T07:42:03Z,2025-01-03T02:39:51Z,None,https://github.com/risingwavelabs/risingwave/p...,[7934],[7934],[],[7934]
19,2334613112,8523,https://api.github.com/repos/BerriAI/litellm,krrishdholakia,17561003,Human,LiteLLM Improvements (02/13/2025) p1,* fix(router.py): add more deployment timeout ...,closed,2025-02-13T19:48:48Z,2025-02-14T01:10:22Z,2025-02-14T01:10:22Z,https://github.com/BerriAI/litellm/pull/8523,[],[],[8507],[8507]
21,2313919432,8227,https://api.github.com/repos/BerriAI/litellm,krrishdholakia,17561003,Human,fix(openai/): allows 'reasoning_effort' param ...,## Title \r\n\r\nBumps openai dependency to al...,closed,2025-02-04T00:14:19Z,2025-02-04T06:39:10Z,2025-02-04T06:39:10Z,https://github.com/BerriAI/litellm/pull/8227,[],[],[8217],[8217]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9867,3152714709,3020,https://api.github.com/repos/crewAIInc/crewAI,devin-ai-integration[bot],158243242,Devin,Fix type annotation mismatch in Task.context f...,# Fix type annotation mismatch in Task.context...,closed,2025-06-17T09:10:24Z,2025-06-27T18:07:35Z,None,https://github.com/crewAIInc/crewAI/pull/3020,[3019],[3019],[3019],[3019]
9868,2919829243,2373,https://api.github.com/repos/crewAIInc/crewAI,devin-ai-integration[bot],158243242,Devin,Fix #2372: Update OpenTelemetry version constr...,This PR fixes issue #2372 by updating the Open...,closed,2025-03-14T10:28:17Z,2025-03-21T17:28:32Z,None,https://github.com/crewAIInc/crewAI/pull/2373,[2372],[2372],[2372],[2372]
9869,2874930848,2210,https://api.github.com/repos/crewAIInc/crewAI,devin-ai-integration[bot],158243242,Devin,fix: prevent duplicate execution of WebSocket ...,Fixes #2209\n\n## Description\nThis PR fixes a...,closed,2025-02-24T13:15:31Z,2025-03-05T17:15:15Z,None,https://github.com/crewAIInc/crewAI/pull/2210,[2209],[2209],[2209],[2209]
9873,3275195929,3237,https://api.github.com/repos/crewAIInc/crewAI,devin-ai-integration[bot],158243242,Devin,Fix Windows path length issue in memory storage,# Fix Windows path length issue in memory stor...,open,2025-07-29T23:01:35Z,None,None,https://github.com/crewAIInc/crewAI/pull/3237,[3236],[3236],[],[3236]


In [10]:
v1 = pd.read_csv("fixes_with_issues_linked_or_on_body.csv")
v1.rename(columns = {'linked_issues':'issues'}, inplace = True)

In [11]:
v1_keys = v1[['id']].drop_duplicates()
novo_keys = prs_com_issue[['id']].drop_duplicates()

print(f"Total de PRs únicos no v1 (Antigo): {len(v1_keys)}")
print(f"Total de PRs únicos no prs_com_issue (Novo): {len(novo_keys)}")
print("-" * 30)
df_diff = pd.merge(
    v1_keys,
    novo_keys,
    on='id',
    how='outer',
    indicator=True  # <-- Cria a coluna '_merge'
)

print("Análise da Diferença (Contagem de PRs):")
contagem = df_diff['_merge'].value_counts()
print(contagem)
print("-" * 30)

# 1. PRs NOVOS (Encontrados no 'novo' mas NÃO no 'v1')
novos_ids = df_diff[df_diff['_merge'] == 'right_only']['id']
df_novos_encontrados = prs_com_issue[prs_com_issue['id'].isin(novos_ids)]

# 2. PRs PERDIDOS (Estavam no 'v1' mas NÃO no 'novo')
perdidos_ids = df_diff[df_diff['_merge'] == 'left_only']['id']
df_perdidos = v1[v1['id'].isin(perdidos_ids)]

# 3. PRs EM COMUM (Estavam em ambos)
comuns_ids = df_diff[df_diff['_merge'] == 'both']['id']
df_comuns = prs_com_issue[prs_com_issue['id'].isin(comuns_ids)]

Total de PRs únicos no v1 (Antigo): 3043
Total de PRs únicos no prs_com_issue (Novo): 3044
------------------------------
Análise da Diferença (Contagem de PRs):
_merge
both          3043
right_only       1
left_only        0
Name: count, dtype: int64
------------------------------


In [17]:
df_comuns[['id','number','repo_url','issues']]

,id,number,repo_url,issues
1,2265431531,83085,https://api.github.com/repos/getsentry/sentry,[82998]
7,2563038011,76,https://api.github.com/repos/mediar-ai/terminator,[70]
17,2257483414,19989,https://api.github.com/repos/risingwavelabs/ri...,[7934]
19,2334613112,8523,https://api.github.com/repos/BerriAI/litellm,[8507]
21,2313919432,8227,https://api.github.com/repos/BerriAI/litellm,[8217]
...,...,...,...,...
9867,3152714709,3020,https://api.github.com/repos/crewAIInc/crewAI,[3019]
9868,2919829243,2373,https://api.github.com/repos/crewAIInc/crewAI,[2372]
9869,2874930848,2210,https://api.github.com/repos/crewAIInc/crewAI,[2209]
9873,3275195929,3237,https://api.github.com/repos/crewAIInc/crewAI,[3236]


In [26]:
colunas_chave = ['id', 'number', 'repo_url']

print("Preparando o DataFrame de origem (agregando por chaves)...")
df_source = df_comuns.groupby(
    colunas_chave, 
    as_index=False
)['issues'].first()

# (Opcional) Renomear a coluna para evitar conflito no merge
df_source = df_source.rename(columns={'issues': 'common_issues'})


# --- ETAPA 2: Fazer o Merge (Left Join) ---
# 'how='left'' mantém todas as linhas do 'df_com_issues_linkadas'
# e adiciona os dados do 'df_source' onde as chaves batem.

print("Executando o merge (left join)...")
df_final = pd.merge(
    df_com_issues_linkadas,
    df_source,
    on=colunas_chave,
    how='left'
)

# Onde não houve match, a nova coluna 'common_issues' terá 'NaN'.


# --- ETAPA 3: Limpar os NaNs (fillna com []) ---
# Precisamos da nossa função robusta para converter NaNs, listas
# e arrays numpy (do Parquet) em listas Python limpas.

def clean_cell_to_list(x):
    """
    Função robusta que converte qualquer célula para uma lista Python limpa.
    - Converte np.ndarray -> list
    - Converte np.nan/None -> []
    - Mantém list -> list
    """
    if isinstance(x, np.ndarray):
        return x.tolist()
    if isinstance(x, list):
        return x
    return [] # <-- Converte 'NaN' em '[]'

print("Limpando a nova coluna (convertendo NaNs em [])...")
df_final['common_issues'] = df_final['common_issues'].apply(clean_cell_to_list)
df_final['has_issues'] = df_final.apply(lambda x: True if len(x['common_issues']) > 0 else False,axis = 1)
df_final = df_final.rename(columns={'common_issues': 'issues'})
df_final.drop(columns=['possible_issues','matched_issues','linked_issues'],inplace = True)
# --- Verificação ---
print("Operação concluída.")
print(f"Tamanho do DataFrame final: {len(df_final)}")
print("\nVerificando linhas que NÃO tiveram match (devem ter '[]'):")
display(df_final[df_final['issues'].apply(len) == 0].head())

print("\nVerificando linhas que TIVERAM match (devem ter listas):")
df_final[df_final['issues'].apply(len) > 0].head()

Preparando o DataFrame de origem (agregando por chaves)...
Executando o merge (left join)...
Limpando a nova coluna (convertendo NaNs em [])...
Operação concluída.
Tamanho do DataFrame final: 9884

Verificando linhas que NÃO tiveram match (devem ter '[]'):


,id,number,user,user_id,agent,title,body,state,created_at,closed_at,merged_at,repo_url,html_url,issues,has_issues
0,2438086945,88748,iamrajjoshi,33237075,Human,:bug: fix: update how we fetch workflow_id and...,i realized i made a mistake for how i fetch th...,closed,2025-04-03T21:36:59Z,2025-04-04T15:10:57Z,2025-04-04T15:10:57Z,https://api.github.com/repos/getsentry/sentry,https://github.com/getsentry/sentry/pull/88748,[],False
2,2622011651,94465,bukzor,640328,Human,fix(dev): mktemp: too few X's in template,"For maximum compatibility, busybox mktemp requ...",closed,2025-06-26T18:54:10Z,2025-06-26T19:57:23Z,2025-06-26T19:57:23Z,https://api.github.com/repos/getsentry/sentry,https://github.com/getsentry/sentry/pull/94465,[],False
3,2565399631,92785,dashed,139499,Human,fix(billing): Update calculateCategoryPrepaidU...,Closes https://linear.app/getsentry/issue/BIL-...,closed,2025-06-03T22:22:51Z,2025-06-05T18:13:54Z,2025-06-05T18:13:54Z,https://api.github.com/repos/getsentry/sentry,https://github.com/getsentry/sentry/pull/92785,[],False
4,2374801945,86438,brendanhsentry,171613822,Human,fix: copy updates to checkout page,closes https://github.com/getsentry/getsentry/...,closed,2025-03-05T22:39:12Z,2025-03-06T16:57:20Z,2025-03-06T16:57:20Z,https://api.github.com/repos/getsentry/sentry,https://github.com/getsentry/sentry/pull/86438,[],False
5,2613746056,150,divanshu-go,180667632,Human,fix: Error: Cannot find module './config',None,closed,2025-06-24T06:37:17Z,2025-06-24T14:36:07Z,2025-06-24T14:36:07Z,https://api.github.com/repos/mediar-ai/terminator,https://github.com/mediar-ai/terminator/pull/150,[],False



Verificando linhas que TIVERAM match (devem ter listas):


,id,number,user,user_id,agent,title,body,state,created_at,closed_at,merged_at,repo_url,html_url,issues,has_issues
1,2265431531,83085,ArthurKnaus,7033940,Human,fix(org-stats): Require project membership,### Problem\r\n\r\nIf the user is not member o...,closed,2025-01-08T07:47:13Z,2025-01-08T08:49:40Z,2025-01-08T08:49:40Z,https://api.github.com/repos/getsentry/sentry,https://github.com/getsentry/sentry/pull/83085,[82998],True
7,2563038011,76,soheil-01,161890530,Human,fix: reorder selector match patterns to priori...,#70,closed,2025-06-03T08:07:58Z,2025-06-03T15:07:07Z,2025-06-03T15:07:07Z,https://api.github.com/repos/mediar-ai/terminator,https://github.com/mediar-ai/terminator/pull/76,[70],True
17,2257483414,19989,zwang28,70626450,Human,fix(meta): resolve deadlock caused by Hummock ...,I hereby agree to the terms of the [RisingWave...,closed,2025-01-02T07:42:03Z,2025-01-03T02:39:51Z,None,https://api.github.com/repos/risingwavelabs/ri...,https://github.com/risingwavelabs/risingwave/p...,[7934],True
19,2334613112,8523,krrishdholakia,17561003,Human,LiteLLM Improvements (02/13/2025) p1,* fix(router.py): add more deployment timeout ...,closed,2025-02-13T19:48:48Z,2025-02-14T01:10:22Z,2025-02-14T01:10:22Z,https://api.github.com/repos/BerriAI/litellm,https://github.com/BerriAI/litellm/pull/8523,[8507],True
21,2313919432,8227,krrishdholakia,17561003,Human,fix(openai/): allows 'reasoning_effort' param ...,## Title \r\n\r\nBumps openai dependency to al...,closed,2025-02-04T00:14:19Z,2025-02-04T06:39:10Z,2025-02-04T06:39:10Z,https://api.github.com/repos/BerriAI/litellm,https://github.com/BerriAI/litellm/pull/8227,[8217],True


In [27]:
df_final

,id,number,user,user_id,agent,title,body,state,created_at,closed_at,merged_at,repo_url,html_url,issues,has_issues
0,2438086945,88748,iamrajjoshi,33237075,Human,:bug: fix: update how we fetch workflow_id and...,i realized i made a mistake for how i fetch th...,closed,2025-04-03T21:36:59Z,2025-04-04T15:10:57Z,2025-04-04T15:10:57Z,https://api.github.com/repos/getsentry/sentry,https://github.com/getsentry/sentry/pull/88748,[],False
1,2265431531,83085,ArthurKnaus,7033940,Human,fix(org-stats): Require project membership,### Problem\r\n\r\nIf the user is not member o...,closed,2025-01-08T07:47:13Z,2025-01-08T08:49:40Z,2025-01-08T08:49:40Z,https://api.github.com/repos/getsentry/sentry,https://github.com/getsentry/sentry/pull/83085,[82998],True
2,2622011651,94465,bukzor,640328,Human,fix(dev): mktemp: too few X's in template,"For maximum compatibility, busybox mktemp requ...",closed,2025-06-26T18:54:10Z,2025-06-26T19:57:23Z,2025-06-26T19:57:23Z,https://api.github.com/repos/getsentry/sentry,https://github.com/getsentry/sentry/pull/94465,[],False
3,2565399631,92785,dashed,139499,Human,fix(billing): Update calculateCategoryPrepaidU...,Closes https://linear.app/getsentry/issue/BIL-...,closed,2025-06-03T22:22:51Z,2025-06-05T18:13:54Z,2025-06-05T18:13:54Z,https://api.github.com/repos/getsentry/sentry,https://github.com/getsentry/sentry/pull/92785,[],False
4,2374801945,86438,brendanhsentry,171613822,Human,fix: copy updates to checkout page,closes https://github.com/getsentry/getsentry/...,closed,2025-03-05T22:39:12Z,2025-03-06T16:57:20Z,2025-03-06T16:57:20Z,https://api.github.com/repos/getsentry/sentry,https://github.com/getsentry/sentry/pull/86438,[],False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9879,3260452571,1542,devin-ai-integration[bot],158243242,Devin,Fix CSS color assertions in test_lambdas.py,# Fix CSS color assertions in test_lambdas.py\...,closed,2025-07-24T16:14:38Z,2025-07-25T11:29:31Z,2025-07-25T11:29:31Z,https://api.github.com/repos/reflex-dev/reflex...,https://github.com/reflex-dev/reflex-web/pull/...,[],False
9880,2857103111,2151,devin-ai-integration[bot],158243242,Devin,fix: Initialize storage in StringKnowledgeSource,Fixes #2150\n\n## Issue\nStringKnowledgeSource...,closed,2025-02-17T08:16:22Z,2025-02-25T16:39:20Z,None,https://api.github.com/repos/crewAIInc/crewAI,https://github.com/crewAIInc/crewAI/pull/2151,[2150],True
9881,2857279950,8459,devin-ai-integration[bot],158243242,Devin,Add missing OpenSSL TLSEXT status response codes,Fixes the build failure in the OpenSSL coexist...,closed,2025-02-17T09:32:13Z,2025-02-17T09:34:11Z,None,https://api.github.com/repos/wolfSSL/wolfssl,https://github.com/wolfSSL/wolfssl/pull/8459,[],False
9882,2857942945,2,devin-ai-integration[bot],158243242,Devin,fix: improve dark mode input focus and toggle ...,# UI Improvements: Dark Mode Input Focus and T...,closed,2025-02-17T13:57:22Z,2025-02-17T14:40:12Z,2025-02-17T14:40:12Z,https://api.github.com/repos/jina-ai/deepsearc...,https://github.com/jina-ai/deepsearch-ui/pull/2,[],False


In [28]:
df_final.to_parquet(r'output_files\fix_PRs_with_issues.parquet',index = False)

In [29]:
df_novos_encontrados

,id,number,repo_url,user,user_id,agent,title,body,state,created_at,closed_at,merged_at,html_url,possible_issues,matched_issues,linked_issues,issues
3732,3213360758,35756,https://api.github.com/repos/Azure/azure-rest-...,Copilot,198982749,Copilot,Fix namespace parsing in azure-sdk-tools api-v...,The `parse_pyproject` function in `azure-sdk-t...,closed,2025-07-08T18:05:31Z,2025-07-08T18:06:52Z,None,https://github.com/Azure/azure-rest-api-specs/...,[35755],[35755],[],[35755]
